In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
import time

In [2]:
API_KEY = "RGAPI-780cdd5f-e595-4e01-a62b-6c703e44cab2"
GAME_NAME = "X8Scorpio8X"
TAG_LINE = "NA1"  # do not include "#"
REGION = "americas"

def get_puuid_from_riot_id(game_name, tag_line, api_key, region):
    # URL-encode the game name and tag line
    game_name_encoded = quote(game_name)
    tag_line_encoded = quote(tag_line)
    
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name_encoded}/{tag_line_encoded}"
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data["puuid"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, API_KEY, REGION)
if puuid:
    print("PUUID:", puuid)


PUUID: vrfSU4Nh3GYoxHi2hLXg_yOczDqdE8WulZzWDLl2RZu6wtmH-e8i629HTEVbu1Uej1CpJZ3JIfXu3w


In [3]:
PUUID = puuid
REGION = "americas"  # match routing region

def get_match_ids(puuid, api_key, region, count=20):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count  # number of matches to retrieve (max 100 per call)
    }
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # list of match IDs
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
match_ids = get_match_ids(PUUID, API_KEY, REGION)
if match_ids:
    print("Recent Match IDs:", match_ids)


Recent Match IDs: ['NA1_5294868369', 'NA1_5294817723', 'NA1_5294744470', 'NA1_5294452175', 'NA1_5294441078', 'NA1_5294208274', 'NA1_5293511416', 'NA1_5293497043', 'NA1_5292384975', 'NA1_5292216464', 'NA1_5286912737', 'NA1_5286890246', 'NA1_5286211608', 'NA1_5285777339', 'NA1_5285766604', 'NA1_5285751856', 'NA1_5285733419', 'NA1_5285225464', 'NA1_5285181083', 'NA1_5284678454']


In [4]:
def parse_match_data(match_data):
    if match_data['info']['queueId'] != 420:  # Only solo/duo ranked games
        return []

    teams = match_data['info']['teams']
    participants = match_data['info']['participants']

    # Split participants by teamId
    team1_id = teams[0]['teamId']
    team2_id = teams[1]['teamId']
    team1_players = [p for p in participants if p['teamId'] == team1_id]
    team2_players = [p for p in participants if p['teamId'] == team2_id]

    def team_stats(team, players, other_players):
        def get_ping_sum(key):
            return sum(p.get(key, 0) for p in players)

        return {
            'matchId': match_data['metadata']['matchId'],
            'teamId': team['teamId'],
            'win': team['win'],
            'baronKills': team['objectives']['baron']['kills'],
            'dragonKills': team['objectives']['dragon']['kills'],
            'riftHeraldKills': team['objectives']['riftHerald']['kills'],
            'totalGold': sum(p['goldEarned'] for p in players),
            'goldSpent': sum(p['goldSpent'] for p in players),
            'totalKills': sum(p['kills'] for p in players),
            'totalDeaths': sum(p['deaths'] for p in players),
            'totalTurretsTaken': sum(p.get('challenges', {}).get('turretTakedowns', 0) for p in players),
            'totalInhibitorsTaken': sum(p.get('challenges', {}).get('inhibitorTakedowns', 0) for p in players),
            'totalEliteMonstersTaken': sum(p.get('challenges', {}).get('eliteMonsterKills', 0) for p in players),
            'averageChampionExp': sum(p['champExperience'] for p in players) / len(players),
            'wardsPlaced': sum(p['wardsPlaced'] for p in players),
            'goldDiff': sum(p['goldEarned'] for p in players) - sum(p['goldEarned'] for p in other_players),
            'allInPings': get_ping_sum('allInPings'),
            'assistMePings': get_ping_sum('assistMePings'),
            'commandPings': get_ping_sum('commandPings'),
            'enemyMissingPings': get_ping_sum('enemyMissingPings'),
            'enemyVisionPings': get_ping_sum('enemyVisionPings'),
            'holdPings': get_ping_sum('holdPings'),
            'getBackPings': get_ping_sum('getBackPings'),
            'totalPings': (
                get_ping_sum('allInPings') +
                get_ping_sum('assistMePings') +
                get_ping_sum('commandPings') +
                get_ping_sum('enemyMissingPings') +
                get_ping_sum('enemyVisionPings') +
                get_ping_sum('holdPings') +
                get_ping_sum('getBackPings')
            )
        }


    return [team_stats(teams[0], team1_players, team2_players),
            team_stats(teams[1], team2_players, team1_players)]


# def collect_match_stats(match_ids, api_key):
#     all_rows = []
#     for match_id in match_ids:
#         url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
#         headers = {
#             "X-Riot-Token": api_key
#         }
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             match_data = response.json()
#             rows = parse_match_data(match_data)
#             all_rows.extend(rows)
#         else:
#             print(f"Failed to retrieve match {match_id}: {response.status_code}")
#     return pd.DataFrame(all_rows)

def collect_match_stats(match_ids, api_key):
    all_rows = []
    for i, match_id in enumerate(match_ids):
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {"X-Riot-Token": api_key}
        
        while True:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                match_data = response.json()
                rows = parse_match_data(match_data)
                all_rows.extend(rows)
                break
            elif response.status_code == 429:
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)
            else:
                print(f"Failed to retrieve match {match_id}: {response.status_code}")
                break

        time.sleep(1.3)

    return pd.DataFrame(all_rows)



In [ ]:
df = collect_match_stats(match_ids, api_key=API_KEY)

KeyboardInterrupt: 

In [ ]:
df.to_csv("X8Scorpio8X_Data")

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,wardsPlaced,goldDiff,allInPings,assistMePings,commandPings,enemyMissingPings,enemyVisionPings,holdPings,getBackPings,totalPings
0,NA1_5294452175,100,True,2,4,1,73652,65225,35,25,...,55,2374,1,3,38,21,2,0,6,71
1,NA1_5294452175,200,False,0,0,0,71278,65360,25,35,...,64,-2374,0,10,21,24,5,0,1,61
2,NA1_5294441078,100,True,0,1,0,48742,44350,36,13,...,35,16945,0,0,8,9,0,0,0,17
3,NA1_5294441078,200,False,0,1,0,31797,29750,13,36,...,23,-16945,0,2,16,7,4,0,2,31
4,NA1_5294208274,100,True,1,3,0,90750,79683,52,36,...,68,4808,0,14,32,8,5,0,0,59
5,NA1_5294208274,200,False,1,4,1,85942,85923,36,52,...,95,-4808,2,9,36,27,2,0,3,79
6,NA1_5293511416,100,True,2,2,1,86240,78250,41,41,...,67,10197,1,14,16,17,14,0,8,70
7,NA1_5293511416,200,False,0,4,0,76043,71310,41,42,...,69,-10197,1,18,28,4,2,0,10,63
8,NA1_5293497043,100,False,0,0,0,30417,26500,4,28,...,37,-16748,1,5,5,9,2,0,0,22
9,NA1_5293497043,200,True,0,3,1,47165,39475,28,4,...,42,16748,0,3,12,10,13,0,4,42


Collecting data from people who are in silver rank

In [6]:
names_taglines = {"Paro":"0820", "MicDonuld":"NA1", "prohanp":"NA1", "eGirl Trainer": "queer", "iSejuani":"NA1", "Fartgas12":"NA1", "Guymandudemanguy": "Ocean", "7hunder3":"Lucky", "WHITEsmoke96":"NA1", "grinder enjoyer":"nocap", 
                  "Atomic":"8541", "LucQie":"NA1", "Konlen":"NA1", "Hide Inside Lux":"NA1", "Valdivian":"NA1", "Gray Goo":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=20))

match_ids = np.array(match_ids).flatten().tolist()

In [7]:
# collecting match stats
silver_data = collect_match_stats(match_ids, API_KEY)

In [8]:
silver_data.to_csv("silver_data")
silver_data

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,wardsPlaced,goldDiff,allInPings,assistMePings,commandPings,enemyMissingPings,enemyVisionPings,holdPings,getBackPings,totalPings
0,NA1_5290905508,100,True,0,2,0,32599,28650,24,13,...,29,8200,12,8,19,19,3,0,1,62
1,NA1_5290905508,200,False,0,0,0,24399,20458,12,25,...,21,-8200,0,0,1,12,13,0,0,26
2,NA1_5290901967,100,True,0,0,0,3010,2500,0,1,...,1,-271,0,0,0,1,0,0,0,1
3,NA1_5290901967,200,False,0,0,0,3281,2000,1,0,...,1,271,0,0,1,0,0,0,0,1
4,NA1_5290883199,100,True,0,3,1,47251,41605,31,11,...,27,13805,1,2,8,9,4,0,5,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,NA1_5281683648,200,False,0,1,0,59279,56700,24,47,...,61,-12320,1,2,9,21,11,0,3,47
326,NA1_5278373384,100,True,1,5,1,91198,80225,51,52,...,82,6751,2,17,40,30,42,0,11,142
327,NA1_5278373384,200,False,1,2,0,84447,76933,52,51,...,88,-6751,1,7,33,35,8,0,11,95
328,NA1_5278349815,100,True,1,3,0,79847,73658,41,42,...,88,6149,0,14,18,19,17,0,26,94
